In [ ]:
!pip install tqdm
!pip install seaborn

In [ ]:
!pip install tf-keras-vis
!git clone https://github.com/WittmannF/LRFinder

In [ ]:
import os
import sys
import math
import json
import shutil
import pathlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tqdm import tqdm, tqdm_notebook

sns.set()
%matplotlib inline
from LRFinder.keras_callback import LRFinder

### Download & Prepare Training Dataset

In [ ]:
# Dataset split ratios
np.random.seed(42)
VAL_PERC = 0.10
TEST_PERC = 0.10
TRAIN_PERC = 0.80

# Split each folder into train, val and test sets
base = pathlib.Path("./data/")
base_dirs = [x for x in base.iterdir() if x.is_dir()]
labels = [x.stem for x in base.iterdir() if x.is_dir()]
base_dirs, labels

# Create directory structure
dirs = ['train', 'val', 'test']
print(dirs)
CLASS_LABELS = np.unique(labels)
print(CLASS_LABELS)

for dirname in dirs:
  cur_dir = base / dirname
  # If old data exists, delete it and create a new blank directory
  if cur_dir.is_dir():
    shutil.rmtree(cur_dir)
    print("DIR DELETED:", cur_dir)
  os.makedirs(cur_dir)
  for label in CLASS_LABELS:
    os.makedirs(cur_dir / label)
    print("DIR MADE FOR", cur_dir / label)

In [ ]:
print(base_dirs)

In [ ]:
# Create data splits and move files
for dirname in base_dirs:
  images = [x for x in dirname.iterdir() if x.suffix == ".png"]
  print(images)
  labels = len(images) * [dirname.stem]
  images, labels = np.array(images), np.array(labels)
  # Calculate dataset size for training, validation and test set
  train_size = int(len(images) * TRAIN_PERC)
  val_size = int(len(images) * VAL_PERC)
  test_size = int(len(images) * TEST_PERC)
  print("[INFO] Split size for: ", dirname, ":", train_size, val_size, test_size, 
        (train_size + val_size + test_size))

  # Create a random split of files
  indices = np.arange(len(labels))
  np.random.shuffle(indices)
  images_train, labels_train = images[indices[:train_size]], labels[indices[:train_size]]
  images_val, labels_val = images[indices[train_size:train_size + val_size]], labels[indices[train_size:train_size + val_size]]
  images_test, labels_test = images[indices[train_size + val_size:]], labels[indices[train_size + val_size:]]

  # Copy files for each directory
  # Train Set
  for image, label in zip(images_train, labels_train):
    shutil.copyfile(image, base / "train" / label / image.name)

  # Val Set
  for image, label in zip(images_val, labels_val):
    shutil.copyfile(image, base / "val" / label / image.name)

  # Test Set
  for image, label in zip(images_test, labels_test):
    shutil.copyfile(image, base / "test" / label / image.name)

In [ ]:
# Cleanup old directories
for dirname in base_dirs:
  # If old data exists, delete it
  if dirname.is_dir():
    shutil.rmtree(dirname)
    print("DIR DELETED:", dirname)

### Validate Dataset and Augmentation

In [ ]:
META_SRC = pathlib.Path("./data/")
CKPT_DIR = pathlib.Path("./model_runs/")

# Params / Hyperparams (Model specific)
BATCH_SIZE = 64
NUM_EPOCHS = 25
LEARNING_RATE = 1e-5
DROPOUT_RATE = 0.5
IMG_HEIGHT = 240
IMG_WIDTH = 240
NUM_CHANNELS = 3
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)

In [ ]:
# Create image dataset from directory.
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=(META_SRC / "train"), labels="inferred", label_mode="categorical",
    image_size=(IMG_HEIGHT, IMG_WIDTH), shuffle=True)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=(META_SRC / "val"), labels="inferred", label_mode="categorical", 
    image_size=(IMG_HEIGHT, IMG_WIDTH), shuffle=True)
print("DATASET TYPES: ", type(train_ds), type(val_ds))

"""
# Compute steps per epoch for training and validation datasets.
compute_steps_per_epoch = lambda x: int(math.ceil(1. * (x / BATCH_SIZE)))
# train_steps = compute_steps_per_epoch(tf.size(train_ds))
# val_steps = compute_steps_per_epoch(tf.size(val_ds))
print("DATASET CARDINALITY, TRAIN: {}, VAL: {}".format(
    tf.data.experimental.cardinality(train_ds).numpy(), 
    tf.data.experimental.cardinality(val_ds).numpy()))
train_steps = compute_steps_per_epoch(
    tf.data.experimental.cardinality(train_ds).numpy())
val_steps = compute_steps_per_epoch(
    tf.data.experimental.cardinality(val_ds).numpy())
print("STEPS PER EPOCH, TRAIN: {}, VAL: {}".format(train_steps, val_steps))
"""

# Prefetch the dataset for improved performance.
train_ds.cache().prefetch(tf.data.experimental.AUTOTUNE)
val_ds.cache().prefetch(tf.data.experimental.AUTOTUNE)

# Define Image Augmentation Strategies
img_augmentation = tf.keras.models.Sequential(
  [
   tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.05),
   tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal")
  ],
  name="img_augmentation",
)

In [ ]:
# View Image Samples
plt.figure(figsize=(10, 10))
for (image, label) in train_ds.take(1):
  for i in range(0, 9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title("{}".format(label[i]))
    plt.axis("off")

In [ ]:
# Visualize augmentation results
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[4]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = img_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("uint8"))
        plt.axis("off")

### Build **NW**

In [ ]:
# Imports
import os
import time
import logging
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


class HouseDetectorNetwork:
    def __init__(self, network):
        # Define logger.
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        # self.logger = config.start_logging_to_stdout(self.logger)
        # self.logger = config.start_logging_to_file(self.logger)

        # Define EfficientNet options.
        self.image_res_list = [224, 240, 260, 300, 380, 456, 528, 600]
        self.network_list = [
            "EfficientNetB0",
            "EfficientNetB1",
            "EfficientNetB2",
            "EfficientNetB3",
            "EfficientNetB4",
            "EfficientNetB5",
            "EfficientNetB6",
            "EfficientNetB7",
        ]
        self.levels_list = [
            ["top"],
            ["top", "block7"],
            ["top", "block7", "block6"],
            ["top", "block7", "block6", "block5"],
            ["top", "block7", "block6", "block5", "block4"],
        ]

        # Load config/environment variables.
        # self.ds_path = pathlib.Path(os.getenv("DATASET_PATH"))
        # self.num_classes = os.getenv("DATASET_NUM_CLASSES")
        self.ds_path = pathlib.Path("./data")
        timestr = time.strftime("%Y%m%d")
        self.ckpt_path = pathlib.Path("./model_runs") / timestr
        self.ckpt_path.mkdir(parents=True, exist_ok=True)
        label_list = [["0","1","2","3","4"]]
        self.ds_labels = label_list[0]
        self.num_classes = len(self.ds_labels)

        # Define network.
        # network_index = int(os.getenv("NETWORK_INDEX"))
        self.model_name = self.network_list[network]
        self.IMG_HEIGHT = self.IMG_WIDTH = self.image_res_list[network]

        # Load default hyperparameters.
        self.BATCH_SIZE = 64
        self.NUM_EPOCHS = 50
        self.LEARNING_RATE = 1e-5
        self.DROPOUT_RATE = 0.5
        self.NUM_CHANNELS = 3
        self.INPUT_SHAPE = (self.IMG_HEIGHT, self.IMG_WIDTH, self.NUM_CHANNELS)

        # model placeholder.
        self.model = None

    def make_datasets(self, type):
        print(self.ds_path / type)
        dataset = tf.keras.preprocessing.image_dataset_from_directory(
            directory=(self.ds_path / type),
            labels="inferred",
            label_mode="categorical",
            batch_size=self.BATCH_SIZE,
            image_size=(self.IMG_HEIGHT, self.IMG_WIDTH),
            shuffle=True,
        )
        dataset.cache().prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

    def get_aug(self):
        # Define Image Augmentation Strategies
        img_augmentation = tf.keras.models.Sequential(
            [
                tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.05),
                tf.keras.layers.experimental.preprocessing.RandomFlip(
                    mode="horizontal"
                ),
            ],
            name="img_augmentation",
        )
        print("Image Augmentation strategy defined.")
        return img_augmentation

    def build_and_compile_model(self, ft=True, levels=1):
        # Define model inputs.
        inputs = tf.keras.layers.Input(self.INPUT_SHAPE)
        # Apply data augmentation to inputs.
        x = self.get_aug()(inputs)

        # Load base model from Keras Applications.
        base_model = getattr(tf.keras.applications, self.model_name)(
            include_top=False, input_tensor=x, weights="imagenet"
        )
        # Freeze base model.
        base_model.trainable = False

        # Unfreeze parts of model.
        if ft:
            # Set trainable layers, but keep batch norm layers frozen.
            for layer in base_model.layers:
                if all(
                    lname not in layer.name for lname in self.levels_list[levels]
                ) or isinstance(layer, tf.keras.layers.BatchNormalization):
                    layer.trainable = False
                else:
                    layer.trainable = True

        # Rebuild classifier.
        x = tf.keras.layers.GlobalAveragePooling2D(name="tl_avgpool")(base_model.output)
        x = tf.keras.layers.Dropout(self.DROPOUT_RATE, name="tl_dropout")(x)
        outputs = tf.keras.layers.Dense(
            self.num_classes, activation="softmax", name="tl_pred"
        )(x)

        # Compile model.
        model = tf.keras.Model(inputs, outputs, name=self.model_name)
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.LEARNING_RATE)
        model.compile(
            optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
        )
        print("Model {} successfully compiled.".format(self.model_name))
        self.model = model

    def show_trainable_layers(self):
        print("Trainable layers for model {} are:".format(self.model_name))
        for layer in self.model.layers:
            if layer.trainable:
                print(
                    "LAYER: {}, TRAINABLE: {}, I/P SHAPE: {}, O/P SHAPE: {}".format(
                        layer.name,
                        layer.trainable,
                        layer.input_shape,
                        layer.output_shape,
                    )
                )

    def train_model(self, train_ds, val_ds, levels):
        checkpoint_name = (
            "house_detector."
            + self.model_name
            + "-LEV-"
            + str(levels)
            + ".weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5"
        )
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=self.ckpt_path / checkpoint_name,
            save_weights_only=True,
            monitor="val_loss",
            mode="min",
            save_best_only=True,
            verbose=1,
        )
        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
              # Stop training when `val_loss` is no longer improving
              monitor="val_loss",
              # "no longer improving" being defined as "no better than 1e-2 less"
              min_delta=1e-3,
              # "no longer improving" being further defined as "for at least 2 epochs"
              patience=3,
              verbose=1,
        )
        callbacks = [model_checkpoint_callback, early_stopping_callback]
        print(
            "Starting model training for {} epochs.".format(self.NUM_EPOCHS)
        )
        history = self.model.fit(
            train_ds,
            epochs=self.NUM_EPOCHS,
            callbacks=callbacks,
            validation_data=val_ds,
            verbose=1,
        )
        print("Model training finished.")
        return history

    def save_hist_plot(self, hist, metric):
        timestr = time.strftime("%Y%m%d-%H%M%S")
        plot_name = (
            "house_detector."
            + self.model_name 
            + "-LEV-"
            + str(levels) 
            + "." + metric 
            + "." + timestr + ".png"
        )
        plt.plot(hist.history[metric])
        plt.plot(hist.history["val_" + metric])
        plt.title("model " + metric)
        plt.ylabel(metric)
        plt.xlabel("epoch")
        plt.legend(["train", "validation"], loc="best")
        plt.savefig(self.ckpt_path / plot_name, dpi=1000)
        print("Model plot for {} saved.".format(metric))
        plt.show()

    def evaluate_model(self, test_ds):
        print("Starting model evaluation on test set.")
        result = self.model.evaluate(test_ds)
        print(dict(zip(self.model.metrics_names, result)))

        y_true = list()
        y_pred = list()
        for i, l in test_ds.unbatch():
            y_pred.append(self.model.predict(np.expand_dims(i, axis=0)))
            y_true.append(l.numpy())

        for i in range(len(self.ds_labels)):
            self.show_per_class_accuracy(y_true, y_pred, i)

    def show_per_class_accuracy(self, y_true, y_pred, class_num):
        cnt = 0
        for yt, yp in zip(y_true, y_pred):
            yt, yp = np.argmax(yt), np.argmax(yp)
            if yt == class_num == yp:
                cnt += 1
        print(
            "Accuracy for class {} is {}%".format(class_num, (cnt / len(y_true)) * 100 * self.num_classes)
        )

### Train & Evaluate Network

In [ ]:
network = 3
levels = 0

# Make in memory datasets from images to train the network.
house_detector_network = HouseDetectorNetwork(network)

train_ds = house_detector_network.make_datasets("train")
val_ds = house_detector_network.make_datasets("val")
test_ds = house_detector_network.make_datasets("test")

# Build House Detector model
house_detector_network.build_and_compile_model(True, levels)
# Check trainable layers.
house_detector_network.show_trainable_layers()

In [ ]:
# Train model.
history = house_detector_network.train_model(train_ds, val_ds, levels)
# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")
b1_0 = history



In [ ]:
with open("./plot_history/b2_LEV0_Net3.json", "w") as f:
    json.dump(b1_0.history, f)

# Evaluate model.
house_detector_network.evaluate_model(test_ds)

# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")

In [ ]:
network = 3
levels = 1

# Make in memory datasets from images to train the network.
house_detector_network = HouseDetectorNetwork(network)

train_ds = house_detector_network.make_datasets("train")
val_ds = house_detector_network.make_datasets("val")
test_ds = house_detector_network.make_datasets("test")

# Build House Detector model
house_detector_network.build_and_compile_model(True, levels)
# Check trainable layers.
house_detector_network.show_trainable_layers()

# Train model.
history = house_detector_network.train_model(train_ds, val_ds, levels)
# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")
b1_0 = history

with open("./plot_history/b2_LEV1_Net3.json", "w") as f:
    json.dump(b1_0.history, f)

# Evaluate model.
house_detector_network.evaluate_model(test_ds)

# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")

In [ ]:
network = 3
levels = 2

# Make in memory datasets from images to train the network.
house_detector_network = HouseDetectorNetwork(network)

train_ds = house_detector_network.make_datasets("train")
val_ds = house_detector_network.make_datasets("val")
test_ds = house_detector_network.make_datasets("test")

# Build House Detector model
house_detector_network.build_and_compile_model(True, levels)
# Check trainable layers.
house_detector_network.show_trainable_layers()

# Train model.
history = house_detector_network.train_model(train_ds, val_ds, levels)
# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")
b1_0 = history

with open("./plot_history/b2_LEV2_Net3.json", "w") as f:
    json.dump(b1_0.history, f)

# Evaluate model.
house_detector_network.evaluate_model(test_ds)

# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")

In [ ]:
network = 3
levels = 3

# Make in memory datasets from images to train the network.
house_detector_network = HouseDetectorNetwork(network)

train_ds = house_detector_network.make_datasets("train")
val_ds = house_detector_network.make_datasets("val")
test_ds = house_detector_network.make_datasets("test")

# Build House Detector model
house_detector_network.build_and_compile_model(True, levels)
# Check trainable layers.
house_detector_network.show_trainable_layers()

# Train model.
history = house_detector_network.train_model(train_ds, val_ds, levels)
# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")
b1_0 = history

with open("./plot_history/b2_LEV3_Net3.json", "w") as f:
    json.dump(b1_0.history, f)

# Evaluate model.
house_detector_network.evaluate_model(test_ds)

# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")

In [ ]:
network = 3
levels = 4

# Make in memory datasets from images to train the network.
house_detector_network = HouseDetectorNetwork(network)

train_ds = house_detector_network.make_datasets("train")
val_ds = house_detector_network.make_datasets("val")
test_ds = house_detector_network.make_datasets("test")

# Build House Detector model
house_detector_network.build_and_compile_model(True, levels)
# Check trainable layers.
house_detector_network.show_trainable_layers()

# Train model.
history = house_detector_network.train_model(train_ds, val_ds, levels)
# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")
b1_0 = history

with open("./plot_history/b2_LEV4_Net3.json", "w") as f:
    json.dump(b1_0.history, f)

# Evaluate model.
house_detector_network.evaluate_model(test_ds)

# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")

In [ ]:
network = 1
levels = 1
ds_num = 0

# Make in memory datasets from images to train the network.
house_detector_network = HouseDetectorNetwork(network, ds_num)
train_ds = house_detector_network.make_datasets("train")
val_ds = house_detector_network.make_datasets("val")
test_ds = house_detector_network.make_datasets("test")

# Build House Detector model
house_detector_network.build_and_compile_model(True, levels)
# Check trainable layers.
# house_detector_network.show_trainable_layers()
# Train model.
history = house_detector_network.train_model(train_ds, val_ds, levels)

# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")
b1_1 = history

with open("/content/b1_1.json", "w") as f:
    json.dump(b1_1.history, f)


# Evaluate model.
house_detector_network.evaluate_model(test_ds)

In [ ]:
# b1_1.history

In [ ]:
network = 1
levels = 2
ds_num = 0

# Make in memory datasets from images to train the network.
house_detector_network = HouseDetectorNetwork(network, ds_num)
train_ds = house_detector_network.make_datasets("train")
val_ds = house_detector_network.make_datasets("val")
test_ds = house_detector_network.make_datasets("test")

# Build House Detector model
house_detector_network.build_and_compile_model(True, levels)
# Check trainable layers.
# house_detector_network.show_trainable_layers()
# Train model.
history = house_detector_network.train_model(train_ds, val_ds, levels)

# Save Accuracy Plots.
house_detector_network.save_hist_plot(history, "accuracy")
house_detector_network.save_hist_plot(history, "loss")
b1_2 = history

with open("/content/b1_2.json", "w") as f:
    json.dump(b1_2.history, f)

# Evaluate model.
house_detector_network.evaluate_model(test_ds)

In [ ]:
b1_2.history

### Visualisations

In [ ]:
def smooth(scalars, weight):  # Weight between 0 and 1
    last = scalars[0]  # First value in the plot (first timestep)
    smoothed = list()
    for point in scalars:
        smoothed_val = last * weight + (1 - weight) * point  # Calculate smoothed value
        smoothed.append(smoothed_val)                        # Save it
        last = smoothed_val  

In [ ]:
b1_LEV4_Net0=json.load(open('./plot_history/b1_LEV4_Net0.json', "rb"))
b1_LEV4_Net1=json.load(open('./plot_history/b1_LEV4_Net1.json', "rb"))
b1_LEV4_Net2=json.load(open('./plot_history/b1_LEV4_Net2.json', "rb"))
b1_LEV4_Net3=json.load(open('./plot_history/b1_LEV4_Net3.json', "rb"))
b1_LEV4_Net4=json.load(open('./plot_history/b1_LEV4_Net4.json', "rb"))

sns.set_palette("muted")
sns.set_style("whitegrid")

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
fig.suptitle("Training different variants of {}".format("EfficientNet"))
plt.subplots_adjust(top=0.925)

axes[0][0].set_title("Training Loss")
axes[0][1].set_title("Validation Loss")
axes[1][0].set_title("Training Accuracy")
axes[1][1].set_title("Validation Accuracy")

epochs_3 = [i for i in range(1, len(b1_LEV4_Net3['loss']) + 1)]
epochs_4 = [i for i in range(1, len(b1_LEV4_Net4['loss']) + 1)]
epochs_2 = [i for i in range(1, len(b1_LEV4_Net2['loss']) + 1)]
epochs_1 = [i for i in range(1, len(b1_LEV4_Net1['loss']) + 1)]
epochs_0 = [i for i in range(1, len(b1_LEV4_Net0['loss']) + 1)]


plot = sns.lineplot(ax=axes[0][0], x=epochs_0, 
                    y=b1_LEV4_Net0['loss'], 
                    label="B0")
plot = sns.lineplot(ax=axes[0][0], x=epochs_1, 
                    y=b1_LEV4_Net1['loss'], 
                    label="B1")
plot = sns.lineplot(ax=axes[0][0], x=epochs_2, 
                    y=b1_LEV4_Net2['loss'], 
                    label="B2")
plot = sns.lineplot(ax=axes[0][0], x=epochs_3, 
                    y=b1_LEV4_Net3['loss'], 
                    label="B3")
plot = sns.lineplot(ax=axes[0][0], x=epochs_4, 
                    y=b1_LEV4_Net4['loss'], 
                    label="B4")

axes[0][0].legend()
axes[0][0].set(xlabel="Epochs")
axes[0][0].set(ylabel="Loss")

plot = sns.lineplot(ax=axes[0][1], x=epochs_0, 
                    y=b1_LEV4_Net0['val_loss'], 
                    label="B0")
plot = sns.lineplot(ax=axes[0][1], x=epochs_1, 
                    y=b1_LEV4_Net1['val_loss'], 
                    label="B1")
plot = sns.lineplot(ax=axes[0][1], x=epochs_2, 
                    y=b1_LEV4_Net2['val_loss'], 
                    label="B2")
plot = sns.lineplot(ax=axes[0][1], x=epochs_3, 
                    y=b1_LEV4_Net3['val_loss'], 
                    label="B3")
plot = sns.lineplot(ax=axes[0][1], x=epochs_4, 
                    y=b1_LEV4_Net4['val_loss'], 
                    label="B4")


axes[0][1].legend()
axes[0][1].set(xlabel="Epochs")
axes[0][1].set(ylabel="Val Loss")

plot = sns.lineplot(ax=axes[1][0], x=epochs_0, 
                    y=b1_LEV4_Net0['accuracy'], 
                    label="B0")
plot = sns.lineplot(ax=axes[1][0], x=epochs_1, 
                    y=b1_LEV4_Net1['accuracy'], 
                    label="B1")
plot = sns.lineplot(ax=axes[1][0], x=epochs_2, 
                    y=b1_LEV4_Net2['accuracy'], 
                    label="B2")
plot = sns.lineplot(ax=axes[1][0], x=epochs_3, 
                    y=b1_LEV4_Net3['accuracy'], 
                    label="B3")
plot = sns.lineplot(ax=axes[1][0], x=epochs_4, 
                    y=b1_LEV4_Net4['accuracy'], 
                    label="B4")

axes[1][0].legend()
axes[1][0].set(xlabel="Epochs")
axes[1][0].set(ylabel="Accuracy")

plot = sns.lineplot(ax=axes[1][1], x=epochs_0, 
                    y=b1_LEV4_Net0['val_accuracy'], 
                    label="B0")
plot = sns.lineplot(ax=axes[1][1], x=epochs_1, 
                    y=b1_LEV4_Net1['val_accuracy'], 
                    label="B1")
plot = sns.lineplot(ax=axes[1][1], x=epochs_2, 
                    y=b1_LEV4_Net2['val_accuracy'], 
                    label="B2")
plot = sns.lineplot(ax=axes[1][1], x=epochs_3, 
                    y=b1_LEV4_Net3['val_accuracy'], 
                    label="B3")
plot = sns.lineplot(ax=axes[1][1], x=epochs_4, 
                    y=b1_LEV4_Net4['val_accuracy'], 
                    label="B4")

axes[1][1].legend()
axes[1][1].set(xlabel="Epochs")
axes[1][1].set(ylabel="Val Accuracy")

plt.savefig('Var_EFFNet.png', bbox_inches="tight")

In [ ]:
b2_LEV0_Net3=json.load(open('./plot_history/b2_LEV0_Net3.json', "rb"))
b2_LEV1_Net3=json.load(open('./plot_history/b2_LEV1_Net3.json', "rb"))
b2_LEV2_Net3=json.load(open('./plot_history/b2_LEV2_Net3.json', "rb"))
b2_LEV3_Net3=json.load(open('./plot_history/b2_LEV3_Net3.json', "rb"))
b2_LEV4_Net3=json.load(open('./plot_history/b2_LEV4_Net3.json', "rb"))


sns.set_palette("muted")
sns.set_style("whitegrid")

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
fig.suptitle("Fine-Tuning {}".format("EfficientNetB3"))
plt.subplots_adjust(top=0.925)
axes[0][0].set_title("Training Loss")
axes[0][1].set_title("Validation Loss")
axes[1][0].set_title("Training Accuracy")
axes[1][1].set_title("Validation Accuracy")

# y_labels = ["LOSS", "VAL_LOSS", "ACCU", "VAL_ACCU"]
epochs_3 = [i for i in range(1, len(b2_LEV3_Net3['loss']) + 1)]
epochs_4 = [i for i in range(1, len(b2_LEV4_Net3['loss']) + 1)]
epochs_2 = [i for i in range(1, len(b2_LEV2_Net3['loss']) + 1)]
epochs_1 = [i for i in range(1, len(b2_LEV1_Net3['loss']) + 1)]
epochs_0 = [i for i in range(1, len(b2_LEV0_Net3['loss']) + 1)]

# plot.set_yticklabels(plot.get_yticklabels(), rotation=90, horizontalalignment='center')

plot = sns.lineplot(ax=axes[0][0], x=epochs_0, 
                    y=b2_LEV0_Net3['loss'], 
                    label="TOP")
plot = sns.lineplot(ax=axes[0][0], x=epochs_1, 
                    y=b2_LEV1_Net3['loss'], 
                    label="TOP, (B7)")
plot = sns.lineplot(ax=axes[0][0], x=epochs_2, 
                    y=b2_LEV2_Net3['loss'], 
                    label="TOP, (B6-B7)")
plot = sns.lineplot(ax=axes[0][0], x=epochs_3, 
                    y=b2_LEV3_Net3['loss'], 
                    label="TOP, (B5-B7)")
plot = sns.lineplot(ax=axes[0][0], x=epochs_4, 
                    y=b2_LEV4_Net3['loss'], 
                    label="TOP, (B4-B7)")
axes[0][0].legend()
axes[0][0].set(xlabel="Epochs")
axes[0][0].set(ylabel="Loss")

plot = sns.lineplot(ax=axes[0][1], x=epochs_0, 
                    y=b2_LEV0_Net3['val_loss'], 
                    label="TOP")
plot = sns.lineplot(ax=axes[0][1], x=epochs_1, 
                    y=b2_LEV1_Net3['val_loss'], 
                    label="TOP, (B7)")
plot = sns.lineplot(ax=axes[0][1], x=epochs_2, 
                    y=b2_LEV2_Net3['val_loss'], 
                    label="TOP, (B6-B7)")
plot = sns.lineplot(ax=axes[0][1], x=epochs_3, 
                    y=b2_LEV3_Net3['val_loss'], 
                    label="TOP, (B5-B7)")
plot = sns.lineplot(ax=axes[0][1], x=epochs_4, 
                    y=b2_LEV4_Net3['val_loss'], 
                    label="TOP, (B4-B7)")
axes[0][1].legend()
axes[0][1].set(xlabel="Epochs")
axes[0][1].set(ylabel="Val Loss")

plot = sns.lineplot(ax=axes[1][0], x=epochs_0, 
                    y=b2_LEV0_Net3['accuracy'], 
                    label="TOP")
plot = sns.lineplot(ax=axes[1][0], x=epochs_1, 
                    y=b2_LEV1_Net3['accuracy'], 
                    label="TOP, (B7)")
plot = sns.lineplot(ax=axes[1][0], x=epochs_2, 
                    y=b2_LEV2_Net3['accuracy'], 
                    label="TOP, (B6-B7)")
plot = sns.lineplot(ax=axes[1][0], x=epochs_3, 
                    y=b2_LEV3_Net3['accuracy'], 
                    label="TOP, (B5-B7)")
plot = sns.lineplot(ax=axes[1][0], x=epochs_4, 
                    y=b2_LEV4_Net3['accuracy'], 
                    label="TOP, (B4-B7)")
axes[1][0].legend()
axes[1][0].set(xlabel="Epochs")
axes[1][0].set(ylabel="Accuracy")

plot = sns.lineplot(ax=axes[1][1], x=epochs_0, 
                    y=b2_LEV0_Net3['val_accuracy'], 
                    label="TOP")
plot = sns.lineplot(ax=axes[1][1], x=epochs_1, 
                    y=b2_LEV1_Net3['val_accuracy'], 
                    label="TOP, (B7)")
plot = sns.lineplot(ax=axes[1][1], x=epochs_2, 
                    y=b2_LEV2_Net3['val_accuracy'], 
                    label="TOP, (B6-B7)")
plot = sns.lineplot(ax=axes[1][1], x=epochs_3, 
                    y=b2_LEV3_Net3['val_accuracy'], 
                    label="TOP, (B5-B7)")
plot = sns.lineplot(ax=axes[1][1], x=epochs_4, 
                    y=b2_LEV4_Net3['val_accuracy'], 
                    label="TOP, (B4-B7)")
axes[1][1].legend()
axes[1][1].set(xlabel="Epochs")
axes[1][1].set(ylabel="Val Accuracy")
plt.savefig("./finetuning_EfficientNetB3_layers.png", bbox_inches="tight")

In [ ]:
import matplotlib.image as mpimg
# Annotation Example
sns.set_palette("muted")
# sns.set_style("whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
# fig.suptitle("Images from Preprocessing Dataset")
# plt.subplots_adjust(top=0.975)
plt.tight_layout()
axes[0].set_title("Label 'building'")
axes[1].set_title("Label 'no-building'")

img_h = mpimg.imread('/home/home/house_detector_ds_02/test/house/3IgRzt7pwuiPI-yQz3Sy1Q.jpg')
img_nh = mpimg.imread('/home/home/house_detector_ds_02/test/nohouse/_SZfEqKz2qOJVAW4Xd4UdA.jpg')

axes[0].imshow(img_h)
axes[1].imshow(img_nh)

axes[0].xaxis.set_visible(False)
axes[0].yaxis.set_visible(False)
axes[1].xaxis.set_visible(False)
axes[1].yaxis.set_visible(False)

plt.savefig("/content/03_appen.png", bbox_inches="tight")

### Try LRFinder

In [ ]:
network = 1
levels = 1
ds_num = 0

# Make in memory datasets from images to train the network.
house_detector_network = HouseDetectorNetwork(network, ds_num)
house_detector_network.build_and_compile_model(True, levels)

# Params / Hyperparams (Model specific)
BATCH_SIZE = 32
NUM_EPOCHS = 10
DROPOUT_RATE = 0.25
# IMG_HEIGHT = 240
# IMG_WIDTH = 240
# NUM_CHANNELS = 3
# INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)

train_ds = house_detector_network.make_datasets("train")
val_ds = house_detector_network.make_datasets("val")
test_ds = house_detector_network.make_datasets("test")

# Define Callback
lr_finder = LRFinder(min_lr=1e-5, max_lr=1)

history = house_detector_network.model.fit(
            train_ds,
            epochs=NUM_EPOCHS,
            callbacks=[lr_finder],
            validation_data=val_ds,
            verbose=1,
        )

### Keras Tuner

In [ ]:
!pip install keras-tuner

In [ ]:
print(type(os.path.join(os.getcwd(), "pokesh/path")))
os.path.dirname(os.path.join(os.getcwd(), "pokesh/path"))

In [ ]:
dict(zip([1,2,3], [4,5,6]))

### Use Pretrained Model to Infer

In [ ]:
# Load New Dataset
!unzip -o '/content/drive/My Drive/anupam/datasets/building_age_network/images/scraped/konstanz_images.zip' -d '/home/'

In [ ]:
KONSTANZ_DIR = pathlib.Path("/home/konstanz_images/")
img_width, img_height = IMG_HEIGHT, IMG_WIDTH
# Make list of usable files
konstanz_images = sorted(KONSTANZ_DIR.rglob("*.jpg"))
print(len(konstanz_images))

In [ ]:
import os

num_skipped = 0
for fpath in konstanz_images:
    try:
        fobj = open(fpath, "rb")
        is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
    finally:
        fobj.close()

    if not is_jfif:
        num_skipped += 1
        # Delete corrupted image
        os.remove(fpath)

print("Deleted %d images" % num_skipped)

In [ ]:
# def inferencing_dataset(ds_path, BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH):
#     dataset = tf.keras.preprocessing.image_dataset_from_directory(
#         directory=ds_path,
#         batch_size=BATCH_SIZE * 2,
#         image_size=(IMG_HEIGHT, IMG_WIDTH),
#         label_mode=None
#     )
#     dataset.cache().prefetch(tf.data.experimental.AUTOTUNE)
#     return dataset
# konstanz_dataset = inferencing_dataset(KONSTANZ_DIR, 32, IMG_HEIGHT, IMG_WIDTH)

In [ ]:
def load_test_images(images, img_width, img_height):
  test_images = []
  for i in images:
    img = tf.keras.preprocessing.image.load_img(i, target_size=(img_width, img_height))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    test_images.append(img)
  return test_images
test_images = load_test_images(konstanz_images, img_width, img_height)

In [ ]:
network = 2
levels = 2
ds_num = 0
detection_model = HouseDetectorNetwork(network, ds_num)
detection_model.build_and_compile_model(True, levels)
detection_model.model.load_weights("/content/drive/MyDrive/anupam/model_runs/house_detector_runs/20210323/house_detector.EfficientNetB2-LEV-2.weights.04-0.01-1.00.hdf5")

In [ ]:
# test_ds = detection_model.make_datasets("test")
# # Evaluate model.
# detection_model.evaluate_model(test_ds)

In [ ]:
konstanz_preds = detection_model.model.predict(np.vstack(test_images), batch_size=detection_model.BATCH_SIZE)

In [ ]:
savepreds = {}
for i, p in zip(konstanz_images, konstanz_preds):
    savepreds[i.name] = "house" if np.argmax(p) == 0 else "nohouse"

In [ ]:
# Save predictions to file.
with open("/content/drive/My Drive/anupam/datasets/building_age_network/images/scraped/konstanz_preds.json", "w") as f:
  json.dump(savepreds, f)

In [ ]:
savepreds